# Question 1

In [1]:
import pandas as pd

## Use pandas to read table from wiki

In [60]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [61]:
df = df[df['Borough'] != 'Not assigned']

In [62]:
df['Neighbourhood'] = df.apply(lambda row: row['Borough'] if row['Neighbourhood']=='Not assigned' else row['Neighbourhood'], axis=1)

In [63]:
new_df = pd.DataFrame(columns=['Postcode','Borough','Neighbourhood'])
for code in df['Postcode'].unique():
    Neighbourhood_combined = ','.join(df[df['Postcode']==code]['Neighbourhood'].values)
    Borough = df[df['Postcode']==code]['Borough'].unique()[0]
    new_row = pd.DataFrame(columns=['Postcode','Borough','Neighbourhood'], 
                           data = [[code, Borough, Neighbourhood_combined]])
    new_df = new_df.append(new_row)
new_df.reset_index(drop=True, inplace=True)

## Create new df based on the row df

In [72]:
new_df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


## Check the shape

In [65]:
new_df.shape

(103, 3)

# Question 2

## Geocoder is unstable, I use the geo dataset

In [67]:
geo_data = pd.read_csv('Geospatial_Coordinates.csv')

In [68]:
geo_data.rename({'Postal Code':'Postcode'},axis=1, inplace=True)

## Merge tables to get the dataframe

In [69]:
df = pd.merge(left=new_df,right=geo_data,on='Postcode')

In [70]:
df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


# Question 3

In [74]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

## Generate the map

In [84]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=10)

# add markers to map
new_df = df.drop(columns=['Postcode'])
for lat, lng, borough, neighborhood in zip(new_df['Latitude'], new_df['Longitude'], new_df['Borough'], new_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto